<a href="https://colab.research.google.com/github/AYA0HASSAN/ArabicAbusiveWordDetection/blob/main/Abusive_Word_Detection_V04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers
!pip install farasapy
!pip install pyarabic
!pip install qalsadi
!git clone https://github.com/aub-mind/arabert
!pip install arabert

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached PyArabic-0.6.15-py3-none-any.whl (126 kB)
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached qalsadi-0.4.5-py3-none-any.whl (256 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 KB 18.5 MB/s eta 0:00:00
  Using cached alyahmor-0.1.5-py3-none-any.whl (51 kB)
  Using cached Tashaphyne-0.3.6-py3-none-any.whl (251 kB)
  Using cached pickleDB-0.9.2.tar.gz (3.7 kB)
  Preparing metadata (setup.py) ... done
  Using cached Naftawayh-0.4-py3-none-any.whl (332 kB)
  Using cached arramooz_pysqlite-0.3-py3-none-any.whl (9.2 MB)
  Using cached libqutrub-1.2.4.1-py3-none-any.whl (138 kB)
  Created wheel for pickledb: file

In [2]:
from transformers import AutoConfig, AutoModelForSequenceClassification, AutoTokenizer
from transformers.data.processors import SingleSentenceClassificationProcessor
from transformers import Trainer , TrainingArguments
from transformers.trainer_utils import EvaluationStrategy
from transformers.data.processors.utils import InputFeatures
from transformers import pipeline

In [3]:
import pandas as pd
import numpy as np 
import random
import torch
from typing import List
#from py4j.java_gateway import JavaGateway
from farasa.segmenter import FarasaSegmenter
from qalsadi.lemmatizer import Lemmatizer
from arabert import ArabertPreprocessor
from sklearn.model_selection import train_test_split
farasa_segmenter = FarasaSegmenter(interactive=True)
from sklearn.metrics import (accuracy_score, classification_report,
                             confusion_matrix, f1_score, precision_score,
                             recall_score)

/usr/local/lib/python3.8/dist-packages/urllib3/connectionpool.py:842: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  warnings.warn((


100%|██████████| 241M/241M [00:22<00:00, 10.7MiB/s]


[2023-01-08 19:27:51,142 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [4]:
class Dataset:
    def __init__(
        self,
        name: str,
        train: List[pd.DataFrame],
        test: List[pd.DataFrame],
        label_list: List[str],
    ):
        # name is dataframe name 
        self.name = name
        self.train = train
        self.test = test
        self.label_list = label_list


In [5]:
class AraBertModel(Dataset):
    def __init__(self,text,target,modelName,maxLen,labelMap):
        super(AraBertModel).__init__()
        self.text = text
        self.target = target
        self.tokenizer_name = modelName
        self.tokenizer = AutoTokenizer.from_pretrained(modelName)
        self.maxLen = maxLen
        self.labelMap = labelMap
  
    def __len__(self):
        return len(self.text)
  
    def __getitem__(self,item):
        text = str(self.text[item])
        text = " ".join(text.split())
        encodedTweet = self.tokenizer(
            text ,
            max_length = self.maxLen,
            # add_special_tokens = True ,
            # return_token_type_ids = False,
            # pad_to_max_length = True,
            padding='max_length',
            truncation = True,
            # return_attention_mask = True,
            # return_tensors = 'pt'
            )
        # inputIDs = encodedTweet
        return InputFeatures(**encodedTweet,label = self.labelMap[self.target[item]])
    


In [6]:
datasetName = 'HateVsAbusive'
data_df = pd.read_csv('/content/train.csv',encoding = "UTF-8")
labelList = list(data_df.Class.unique())
data_df.head()

,Tweet,Class
0,وزير الخارجية اللبناني جبران باسيل قال في سلسل...,normal
1,سورية بلد الحضارات تربطها بعلية او بحيوان,normal
2,اخي الحاج اذا شعرت انك محرجا من الانتقادات لتص...,normal
3,ما فيك تعيش بلا ما تكب فتن ليل نهار وبكرة قلهم...,normal
4,هذا البطل الذي قاتل وجاذف بحياته لتحيا انت يا ...,abusive


In [7]:
labelMap = {'normal' : 0, 'abusive' : 1, 'hate' : 2}
# labelMap = { v:index for index, v in enumerate(data_df.Class) }

# data_df['Class'] = data_df['Class'].apply(lambda x: labelMap[x])
# data_df['Class']

In [8]:
labelMap

{'normal': 0, 'abusive': 1, 'hate': 2}

In [9]:
train , test = train_test_split(data_df, test_size=0.2,random_state=42)
data = Dataset(datasetName,train,test,labelList)

### MARBERT

In [10]:
# 
model_name="UBC-NLP/MARBERT"
# arabert_prep = ArabertPreprocessor(model_name=model_name)
# data.train['Tweet'] = data.train['Tweet'].apply(lambda x: arabert_prep.preprocess(x))
# data.train['Tweet']

In [11]:
lemmer = Lemmatizer()
data.train['Tweet'] = data.train['Tweet'].apply(lambda x: lemmer.lemmatize_text(x))
data.train['Tweet']

2086    [جبر, باسيل, جله, تاريخ, واحد, من, أسوأ, وسخ, ...
3106    [يا, شباب, روق, شوي, هاد, كل, ما, نسب, او, نبه...
2865    [لك, طول, عمر, بتشتموا, سعودي, شيوخ, وبتسبو, و...
4244                                   [شوى, هالثقة, نفس]
831                         [جبر, باسيل, هد, من, إسرائيل]
                              ...                        
4426    [خراس, ياششمة, فرنج, خراس, واذا, مابتعرف, شوى,...
466                                            [بطل, علا]
3092    [لا, أحد, واجه, عرب, غير, أسياد, في, طهر, و, ا...
3772    [لبنان, إذا, تحت, مطعم, حتى, لو, سميتو, مطعم, ...
860     [ونى, سليم, فرنج, كلب, يتم, تحضير, وراث, أبي, ...
Name: Tweet, Length: 3740, dtype: object

In [12]:
list(data.train['Tweet'][0:2])

[['جبر',
  'باسيل',
  'جله',
  'تاريخ',
  'واحد',
  'من',
  'أسوأ',
  'وسخ',
  'سياسي',
  'طائفي',
  'سفهاء',
  'رأي',
  'في',
  'لبن',
  'أكثر',
  'نجاسة',
  'غباء',
  'علا'],
 ['يا',
  'شباب',
  'روق',
  'شوي',
  'هاد',
  'كل',
  'ما',
  'نسب',
  'او',
  'نبهدلو',
  'بينبسط',
  'زغرط',
  'كم',
  'عند',
  'اصحاب',
  'متابع',
  'اه']]

In [13]:
train_dataset = AraBertModel(data.train['Tweet'].to_list(),
                             data.train['Class'].to_list(),
                             model_name,
                             maxLen=128,
                             labelMap = labelMap
                             )
test_dataset = AraBertModel(data.test['Tweet'].to_list(),
                             data.test['Class'].to_list(),
                             model_name,
                             maxLen=128,
                             labelMap = labelMap
                            )

Downloading:   0%|          | 0.00/376 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/701 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [14]:
# return a pretrained model ready for classification
def modelInit():
  return AutoModelForSequenceClassification.from_pretrained(model_name, return_dict=True, num_labels=len(labelMap))

# evaluation metric
def compute_metrics(p): #p should be of type EvalPrediction
  preds = np.argmax(p.predictions, axis=1)
  assert len(preds) == len(p.label_ids)
  #print(classification_report(p.label_ids,preds))
  print(confusion_matrix(p.label_ids,preds))
  macro_f1 = f1_score(p.label_ids,preds,average='macro')
  #macro_precision = precision_score(p.label_ids,preds,average='macro')
  macro_recall = recall_score(p.label_ids,preds,average='macro')
  acc = accuracy_score(p.label_ids,preds)
  return {       
      'macro_f1' : macro_f1,
      'accuracy': acc,
      'recall' : macro_recall
  }

def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic=True
    torch.backends.cudnn.benchmark = False

In [15]:
# Training Parameters
training_args = TrainingArguments( 
    output_dir= "./train",    
    adam_epsilon = 1e-8,
    learning_rate = 2e-5,
    fp16 = False, # enable this when using V100 or T4 GPU
    per_device_train_batch_size = 16, # up to 64 on 16GB with max len of 128
    per_device_eval_batch_size = 128,
    gradient_accumulation_steps = 2, # use this to scale batch size without needing more memory
    num_train_epochs= 6,
    warmup_ratio = 0,
    do_eval = True,
    evaluation_strategy = 'epoch',
    save_strategy = 'epoch',
    load_best_model_at_end = True, # this allows to automatically get the best model at the end based on whatever metric we want
    metric_for_best_model = 'macro_f1',
    greater_is_better = True,
    seed = 25
  )

set_seed(training_args.seed)

In [16]:
trainer = Trainer(
    model = modelInit(),
    args = training_args,
    train_dataset = train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics,
)
#start the training
trainer.train()

Downloading:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of the model checkpoint at UBC-NLP/MARBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at U

Epoch,Training Loss,Validation Loss,Macro F1,Accuracy,Recall
1,No log,0.619127,0.569818,0.835470,0.596686
2,No log,0.564776,0.679979,0.822650,0.677099
3,No log,0.493834,0.701664,0.822650,0.705169
4,No log,0.492114,0.724092,0.832265,0.733124
5,0.342600,0.500750,0.722581,0.822650,0.743229
6,0.342600,0.523350,0.719583,0.818376,0.741032


***** Running Evaluation *****
  Num examples = 936
  Batch size = 128


[[553  54   0]
 [ 36 228   0]
 [ 14  50   1]]


...

[[516  77  14]
 [ 23 218  23]
 [  8  21  36]]


Saving model checkpoint to ./train/checkpoint-585
Configuration saved in ./train/checkpoint-585/config.json
Model weights saved in ./train/checkpoint-585/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 936
  Batch size = 128


[[512  81  14]
 [ 23 218  23]
 [  8  21  36]]


Saving model checkpoint to ./train/checkpoint-702
Configuration saved in ./train/checkpoint-702/config.json
Model weights saved in ./train/checkpoint-702/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./train/checkpoint-468 (score: 0.7240919621821847).


TrainOutput(global_step=702, training_loss=0.26478001466843476, metrics={'train_runtime': 581.9494, 'train_samples_per_second': 38.56, 'train_steps_per_second': 1.206, 'total_flos': 1476066273454080.0, 'train_loss': 0.26478001466843476, 'epoch': 6.0})

In [17]:
results = trainer.evaluate()
print(results)

***** Running Evaluation *****
  Num examples = 936
  Batch size = 128


[[519  76  12]
 [ 20 229  15]
 [  6  28  31]]
{'eval_loss': 0.49211427569389343, 'eval_macro_f1': 0.7240919621821847, 'eval_accuracy': 0.8322649572649573, 'eval_recall': 0.7331240103480631, 'eval_runtime': 6.9551, 'eval_samples_per_second': 134.578, 'eval_steps_per_second': 1.15, 'epoch': 6.0}


In [22]:
inv_label_map = inv_label_map = { v:k for k, v in labelMap.items()}
print(inv_label_map)
trainer.model.config.label2id = labelMap
trainer.model.config.id2label = inv_label_map
trainer.save_model("output_MARBERT")
train_dataset.tokenizer.save_pretrained("output_MARBERT")
#copy the model to drive
!cp output_MARBERT /content/drive/MyDrive

Saving model checkpoint to output_MARBERT
Configuration saved in output_MARBERT/config.json


{0: 'normal', 1: 'abusive', 2: 'hate'}


Model weights saved in output_MARBERT/pytorch_model.bin
tokenizer config file saved in output_MARBERT/tokenizer_config.json
Special tokens file saved in output_MARBERT/special_tokens_map.json


cp: -r not specified; omitting directory 'output_MARBERT'


In [23]:
# initialize pipline
pipe = pipeline("sentiment-analysis", model="output_MARBERT", device=0, return_all_scores=True)

loading configuration file output_MARBERT/config.json
Model config BertConfig {
  "_name_or_path": "output_MARBERT",
  "architectures": [
    "BertForSequenceClassification"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "directionality": "bidi",
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "normal",
    "1": "abusive",
    "2": "hate"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "abusive": 1,
    "hate": 2,
    "normal": 0
  },
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 0,
  "pooler_fc_size": 768,
  "pooler_num_attention_heads": 12,
  "pooler_num_fc_layers": 3,
  "pooler_size_per_head": 128,
  "pooler_type": "first_token_transform",
  "position_embedding_type": "absolute",
  "problem_type": "single_label_classificat

In [33]:
pipe(["طيب انت اوى","يا حيوان اخرس"])


[[{'label': 'normal', 'score': 0.46579912304878235},
  {'label': 'abusive', 'score': 0.41235557198524475},
  {'label': 'hate', 'score': 0.12184532731771469}],
 [{'label': 'normal', 'score': 0.014734741300344467},
  {'label': 'abusive', 'score': 0.9373983144760132},
  {'label': 'hate', 'score': 0.047866880893707275}]]

In [39]:
x = pipe("قطر مرادف الارهاب")
pred = max(x[0], key=lambda x:x['score'])
pred['label']

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


'hate'

In [40]:
def predictLabel(tweet):
  x = pipe(tweet)
  pred = max(x[0], key=lambda x:x['score'])
  return str(pred['label'])

In [41]:
text_df = pd.read_csv('/content/test.csv',encoding = "UTF-8")
text_df['predictions'] = text_df['Tweet'].apply(lambda x : predictLabel(x))
text_df.head()

/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1043: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


,Tweet,Class,predictions
0,وحاج حكي مثاليات وامه ومابعرف شو ،اخر هم عندكم...,normal,normal
1,قطر مرادف الارهاب,hate,hate
2,ارمني يوناني يحمل جنسية المانية من مواليد سوري...,hate,abusive
3,يا حمار قمة يعني زبالة,abusive,abusive
4,ما حدا غير يلي خلفتك عاهرة يا منحط يا سافل,abusive,abusive


In [44]:
text_df['Class'][0]

'normal'

In [47]:
cumSum = 0
for i in range(len(text_df['Class'])):
  if text_df.Class[i] == text_df.predictions[i]:
    cumSum += 1
print(f"Test Accuracy = {cumSum/len(text_df['Class'])}")

Test Accuracy = 0.8222222222222222
